<a href="https://colab.research.google.com/github/financieras/scikit-learn/blob/main/r_cuadrado_progresivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis progresivo del coeficiente de determinación $R^2$ con regresión lineal en el California Housing Dataset

## Objetivo del proyecto

El propósito de este pequeño proyecto es **demostrar cómo el coeficiente de determinación $R^2$** en modelos de regresión lineal puede mejorar al ir incorporando progresivamente más variables explicativas (columnas) al modelo.  
Se utiliza un dataset real y típico para regresión, entrenando modelos de regresión lineal simple y luego regresión lineal múltiple, para mostrar el efecto en el valor de $R^2$.

***

## ¿Qué es el California Housing Dataset?

El California Housing Dataset es un conjunto de datos obtenido del censo de 1990 de California, que contiene información a nivel distrital sobre características demográficas y de vivienda.  
Tiene 8 variables numéricas que describen aspectos como ingreso mediano, edad de las casas, promedio de habitaciones, población, entre otras. La variable objetivo es el valor medio de las casas en cada distrito.  
Este dataset es muy usado como ejemplo clásico para regresiones lineales porque tiene suficiente complejidad para ser interesante pero es manejable en tamaño y limpieza.

***

## Breve introducción a scikit-learn (sklearn)

Scikit-learn es una biblioteca de Python para aprendizaje automático que incluye implementaciones eficientes y fáciles de usar de una gran variedad de modelos y herramientas, incluyendo regresión lineal.  
Facilita tareas como: carga y división de datos, preprocesamiento, entrenamiento de modelos, predicción y evaluación, con funciones simples y bien documentadas. Es estándar en data science para experimentos rápidos y prototipos.

***

## ¿Qué es el coeficiente de determinación $R^2$?

El coeficiente de determinación $R^2$ es una métrica utilizada para evaluar la calidad del ajuste de un modelo de regresión.  
Indica la proporción de la variabilidad total de la variable objetivo que es explicada por el modelo.  
- Un valor de $R^2 = 1$ implica un ajuste perfecto (todas las predicciones coinciden exactamente con los valores reales).  
- Un valor cercano a 0 indica que el modelo no explica nada de la variabilidad.  
- Valores negativos pueden indicar que el modelo ajusta peor que simplemente usar la media.

***

## Regresión lineal simple y múltiple, y la relación con $R^2$

- La **regresión lineal simple** utiliza una única variable predictora para predecir la variable objetivo.  
- La **regresión lineal múltiple** implica usar varias variables predictoras al mismo tiempo para mejorar la predicción.  

Generalmente, al añadir más variables explicativas relevantes, el modelo puede capturar mejor la variabilidad del fenómeno estudiado, por lo que el $R^2$ suele aumentar.  
Sin embargo, agregar variables irrelevantes o redundantes puede no mejorar el modelo o producir sobreajuste, por eso es importante una selección adecuada de variables y validación.

***

## Explicación del código usado

El código implementa un bucle que:  
1. Carga el California Housing Dataset.  
2. Divide los datos en conjuntos de entrenamiento y prueba.  
3. Define un orden para las variables predictoras que se van a añadir progresivamente.  
4. Para cada conjunto creciente de variables:  
    - Entrena un modelo de regresión lineal con esas variables.  
    - Evalúa el modelo calculando $R^2$ sobre el conjunto de prueba.  
5. Imprime los resultados en una tabla clara que muestra cómo varía $R^2$ al agregar variables.

Esta aproximación permite ver cómo mejora la capacidad predictiva a medida que se le brinda más información al modelo, desde una regresión lineal simple hasta una múltiple que incluye todas las variables seleccionadas.

***

## Interpretación de los resultados obtenidos

Los resultados muestran claramente que:  
- Con solo la variable "MedInc" (ingreso mediano), el modelo ya explica aproximadamente un 46% de la variabilidad del precio medio ($R^2 ≈ 0.4589$).  
- Al agregar más variables relevantes como "HouseAge" y "AveRooms", $R^2$ incrementa paulatinamente, indicando una mejora en la capacidad explicativa.  
- La inclusión de todas las variables listadas produce un modelo con un $R^2 ≈ 0.5758$, es decir, explica cerca del 58% de la variabilidad en el precio de las casas.  

Este comportamiento es típico: añadir variables relevantes mejora el ajuste y el coeficiente de determinación.  
Sin embargo, la mejora puede ser marginal al añadir variables adicionales conforme estas aportan menos información nueva, y siempre hay que validar para evitar sobreajuste.


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# 1. Cargar el dataset
housing = fetch_california_housing(as_frame=True)
X = housing.data
y = housing.target

# 2. Dividir en train y test (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Crear un pipeline con escalado y regresión lineal múltiple
modelo = make_pipeline(StandardScaler(), LinearRegression())

# 4. Entrenar (fit)
modelo.fit(X_train, y_train)

# 5. Predecir con el modelo entrenado
y_pred = modelo.predict(X_test)

# 6. Calcular R² usando r2_score
r2_funcion = r2_score(y_test, y_pred)

# 7. Calcular R² usando el método .score() del modelo
r2_metodo = modelo.score(X_test, y_test)

# 8. Mostrar resultados
print(f"R² en test (r2_score): \t\t{r2_funcion:.4f}")
print(f"R² en test (método .score): \t{r2_metodo:.4f}")

R² en test (r2_score): 		0.5758
R² en test (método .score): 	0.5758


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. Cargar el dataset California Housing
housing = fetch_california_housing(as_frame=True)
X_full = housing.data
y = housing.target

# Lista ordenada de columnas para añadir progresivamente
feature_order = [
    "MedInc",    # ingreso mediano (importante, suele dar buen R^2 solo)
    "HouseAge",  # edad mediana
    "AveRooms",  # promedio de habitaciones
    "AveBedrms", # promedio de dormitorios
    "Population",# población del bloque
    "AveOccup",  # promedio de personas por hogar
    "Latitude",  # latitud
    "Longitude"  # longitud
]

# Dividir el dataset completo en train y test para evitar filtrado
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

print(f"{'Variables usadas':50} {'R² en test':>39}")
print("-" * 91)

# Ir añadiendo columnas una a una y hacer regresión lineal
for i in range(1, len(feature_order) + 1):
    features = feature_order[:i]

    # Seleccionar únicamente las columnas actuales para train y test
    X_train = X_train_full[features]
    X_test = X_test_full[features]

    # Crear y entrenar el modelo
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predecir y calcular R² en test
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    #print(f"{str(features):98} {r2:10.4f}")
    #print(*features, f"{r2:9.4f}")
    print(f"{', '.join(features):80} {r2:7.4f}")

Variables usadas                                                                R² en test
-------------------------------------------------------------------------------------------
MedInc                                                                            0.4589
MedInc, HouseAge                                                                  0.4941
MedInc, HouseAge, AveRooms                                                        0.4972
MedInc, HouseAge, AveRooms, AveBedrms                                             0.5089
MedInc, HouseAge, AveRooms, AveBedrms, Population                                 0.5090
MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup                       0.5099
MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude             0.5139
MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude  0.5758
